In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 🧠 Self-Supervised Models (SimCLR, MoCo, Rotation, JEPA)

### 1. Pre-training

* Allena l’**encoder una sola volta**, su tutti i pazienti di `train_fold0`.
* Salva i checkpoint (encoder + projector).

### 2. Estrazione delle feature

* Carica il **checkpoint fisso** del pretraining.
* Estrai le feature su tutti i fold (`train_fold{i}`, `val_fold{i}`, `test_holdout`).
* Salva i file come `{model}_features_fold{i}.pt`.

### 3. Linear-Probe per ogni fold `i ∈ [0, ..., N-1]`

* **Train**: allena la testa lineare (LogReg, MLP) sulle feature di `train_fold{i}`.
* **Validation**: usa `val_fold{i}` per:

  * early stopping,
  * salvare il miglior probe,
  * applicare **Temperature Scaling**.
* **Test**: valuta il probe calibrato su `test_holdout`, usando anche **MC-Dropout**.

### 4. Aggregazione finale

* Raccogli i risultati su `test_holdout` per ogni fold.
* Calcola **media ± deviazione standard** per ogni metrica (Accuracy, F1, AUC, ECE, incertezza).
* Questo è il risultato finale del tuo modello SSL + probe.

> 🔒 **Non riaddestrare l’encoder nei fold 1–N.**
> Lo scopo è testarne la **capacità di generalizzazione task-agnostica**, non adattarlo a ogni fold.

---

## 2. 🧪 Supervised & Transfer Learning

Qui **non hai un encoder fisso**. Ogni fold ha il proprio training da zero.

### Per ciascun fold `i ∈ [0, ..., N-1]`

* **Train**: allena l’intero modello (es. ResNet-50) su `train_fold{i}`.
* **Validation**: usa `val_fold{i}` per early-stopping e calibrazione.
* **Test**: valuta sempre su `test_holdout` (o sul val-fold se non esiste un holdout).
* Salva un checkpoint per ogni fold (`supervised_fold{i}.pt`, `transfer_fold{i}.pt`, ecc.).

### Aggregazione finale

* Come per gli SSL, calcola la **media ± deviazione** delle metriche su `test_holdout` per i vari fold.
* Non scegli il modello col miglior punteggio, ma riporti la **media aggregata**.

> ℹ️ **Facoltativo**: puoi evidenziare il fold più vicino alla media, **solo a scopo illustrativo**.

---

## 3. 🔁 Confronto tra pipeline SSL e SL

| Step                   | SSL models (SimCLR, MoCo, ...)  | SL/Transfer models              |
| ---------------------- | ------------------------------- | ------------------------------- |
| **Encoder training**   | Solo su fold0                   | Uno per ogni fold               |
| **Feature extraction** | Uno per ogni fold               | –                               |
| **Probe/classifier**   | Uno per ogni fold               | Uno per ogni fold               |
| **Inference**          | Su `test_holdout` per ogni fold | Su `test_holdout` per ogni fold |
| **Checkpoint**         | Encoder 1×                      | 1× per fold                     |
| **Output finale**      | Media ± std su tutti i fold     | Media ± std su tutti i fold     |

---

## 🎯 Perché questa architettura?

* Nei **modelli SSL**, vogliamo dimostrare che un encoder **generalizza** a nuovi pazienti come **feature extractor**, senza mai essere fine-tuned.
* Nei **modelli supervisionati**, alleniamo da capo su ogni fold per valutare una baseline comparabile (ma meno task-agnostica).

In entrambi i casi:

✅ **Non scegli il miglior fold**,
✅ **Riporti solo le medie cross-fold**,
✅ **Dimostri affidabilità, generalizzazione e riproducibilità**.

---

## 📊 Output finale

Le metriche aggregate sono presentate in una tabella riassuntiva:

| Model             | Accuracy (μ±σ) | Macro-F1 (μ±σ) | ECE (μ±σ)   | Uncertainty (μ±σ) |
| ----------------- | -------------- | -------------- | ----------- | ----------------- |
| SimCLR + probe    | 0.83 ± 0.04    | 0.79 ± 0.06    | 0.03 ± 0.01 | 0.12 ± 0.03       |
| MoCo-v2 + probe   | 0.78 ± 0.05    | …              | …           | …                 |
| Rotation + probe  | 0.74 ± 0.08    | …              | …           | …                 |
| Supervised        | 0.80 ± 0.03    | …              | …           | …                 |
| Transfer learning | 0.82 ± 0.04    | …              | …           | …                 |

> Ogni riga rappresenta una valutazione **completa e aggregata** del modello, utile per confronti clinici e accademici.


In [2]:
# ## Cell 1 – Environment Setup & Dependencies
# Compatibile con Google Colab (GPU/CPU) e ambiente locale (VS Code).

# %%
import os, sys, subprocess, importlib.util
from pathlib import Path

print("📦 [DEBUG] Avvio configurazione ambiente…")

# ────────────────────────────────────────────────────────────────────────
# 1) Rileva ambiente (Colab vs locale)
# ────────────────────────────────────────────────────────────────────────
IN_COLAB = Path("/content").exists()
if IN_COLAB:
    print("📍 [DEBUG] Google Colab rilevato.")
    from google.colab import drive  # type: ignore
    drive.mount("/content/drive", force_remount=False)
else:
    print("💻 [DEBUG] Ambiente locale (VS Code / CLI) rilevato.")

# ────────────────────────────────────────────────────────────────────────
# 2) Definisci PROJECT_ROOT (ENV > default mapping)
# ────────────────────────────────────────────────────────────────────────
DEFAULT_ENV_PATHS = {
    "colab": "/content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project",
    "local": "/Users/stefanoroybisignano/Desktop/MLA/project/wsi-ssrl-rcc_project",
}
PROJECT_ROOT = Path(os.getenv(
    "PROJECT_ROOT",
    DEFAULT_ENV_PATHS["colab" if IN_COLAB else "local"])
).resolve()

sys.path.append(str(PROJECT_ROOT / "src"))
print(f"📁 [DEBUG] PROJECT_ROOT → {PROJECT_ROOT}")

# ────────────────────────────────────────────────────────────────────────
# 3) Utility per installare pacchetti mancanti
# ────────────────────────────────────────────────────────────────────────
def _missing(pkgs):
    return [p for p in pkgs if importlib.util.find_spec(p) is None]

def _install(pkgs, idx_url=None):
    if not pkgs:
        return
    cmd = [sys.executable, "-m", "pip", "install", "--quiet"]
    if idx_url:
        cmd += ["--index-url", idx_url]
    subprocess.check_call(cmd + pkgs)

# ────────────────────────────────────────────────────────────────────────
# 4) Verifica / installa PyTorch (se non presente)
#    • In Colab non sovrascrive la versione pre-installata
# ────────────────────────────────────────────────────────────────────────
TORCH_PKGS = ["torch", "torchvision", "torchaudio"]

if _missing(["torch"]):
    print("🔧 [DEBUG] PyTorch non trovato → installazione in corso…")
    if IN_COLAB:
        GPU = Path("/usr/local/cuda").exists()
        INDEX = "https://download.pytorch.org/whl/cu121" if GPU else "https://download.pytorch.org/whl/cpu"
        _install(TORCH_PKGS, INDEX)
    else:  # locale: lascia scegliere all'utente il build corretto
        _install(TORCH_PKGS)
else:
    import torch
    print(f"✅ [DEBUG] PyTorch già presente ({torch.__version__})")

# ────────────────────────────────────────────────────────────────────────
# 5) Installazione pacchetti ausiliari (sempre sicura)
# ────────────────────────────────────────────────────────────────────────
AUX_PKGS = ["webdataset", "tqdm", "pillow", "pyyaml", "joblib"]
missing_aux = _missing(AUX_PKGS)
if missing_aux:
    print(f"🔧 [DEBUG] Installazione pacchetti ausiliari mancanti: {missing_aux}")
    _install(missing_aux)
else:
    print("✅ [DEBUG] Pacchetti ausiliari già presenti.")

# ────────────────────────────────────────────────────────────────────────
# 6) Info dispositivo
# ────────────────────────────────────────────────────────────────────────
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ [DEBUG] Torch device disponibile → {DEVICE}")

# ────────────────────────────────────────────────────────────────────────
# 7) Costante path per Data Tarball (utilizzata negli step successivi)
# ────────────────────────────────────────────────────────────────────────
DATA_TARBALL = PROJECT_ROOT / "data" / "processed"
print(f"📦 [DEBUG] DATA_TARBALL → {DATA_TARBALL}")


📦 [DEBUG] Avvio configurazione ambiente…
📍 [DEBUG] Google Colab rilevato.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 [DEBUG] PROJECT_ROOT → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project
✅ [DEBUG] PyTorch già presente (2.6.0+cu124)
🔧 [DEBUG] Installazione pacchetti ausiliari mancanti: ['pillow', 'pyyaml']
🖥️ [DEBUG] Torch device disponibile → cpu
📦 [DEBUG] DATA_TARBALL → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed


In [3]:
# %% -------------------------------------------------------------------- #
# Cell 2 – Configurazione & Setup Esperimento                            #
# ----------------------------------------------------------------------- #
import yaml
import datetime
import os
import pprint
import random
import numpy as np
import torch
from pathlib import Path

# ─── 1) Carica il file di configurazione ───────────────────────────────
cfg_path = PROJECT_ROOT / "config" / "training.yaml"
assert cfg_path.exists(), f"❌ File mancante: {cfg_path}"
cfg = yaml.safe_load(cfg_path.read_text())
print(f"📄 [DEBUG] Config caricata da: {cfg_path}")

# ─── 2) Genera EXP_CODE ────────────────────────────────────────────────
yaml_exp = cfg.get("exp_code", "")
env_exp  = os.getenv("EXP_CODE", "")
if yaml_exp:
    EXP_CODE, src = yaml_exp, "YAML"
elif env_exp:
    EXP_CODE, src = env_exp, "ENV"
else:
    EXP_CODE, src = datetime.datetime.now().strftime("%Y%m%d%H%M%S"), "TIMESTAMP"
os.environ["EXP_CODE"] = EXP_CODE
cfg["exp_code"] = EXP_CODE
print(f"🔑 [DEBUG] EXP_CODE → {EXP_CODE} (fonte: {src})")

# ─── 3) Parametri dataset & folds ──────────────────────────────────────
DATASET_ID = cfg["data"]["dataset_id"]
FOLDS = cfg.get("folds", [0])
TRAIN_ENCODER_ONCE = cfg.get("train_encoder_once", False)

print(f"🧬 [DEBUG] DATASET_ID         = {DATASET_ID}")
print(f"🔁 [DEBUG] Folds              = {FOLDS}")
print(f"🔒 [DEBUG] train_encoder_once = {TRAIN_ENCODER_ONCE}")

# ─── 4) Crea la directory dell’esperimento ─────────────────────────────
exp_dir_rel = cfg["output"]["exp_dir"].format(
    dataset_id=DATASET_ID,
    exp_code=EXP_CODE
)
# Use PROJECT_ROOT without early resolve to ensure Drive path
EXP_BASE = PROJECT_ROOT / exp_dir_rel
# Safety check: ensure EXP_BASE is under PROJECT_ROOT
if not str(EXP_BASE.resolve()).startswith(str(PROJECT_ROOT.resolve())):
    raise RuntimeError(f"🚨 EXP_BASE NON sotto PROJECT_ROOT! → {EXP_BASE}")
# Create experiment directory
EXP_BASE.mkdir(parents=True, exist_ok=True)
print(f"📂 [DEBUG] EXP_BASE → {EXP_BASE.resolve()}")

# ─── 5) Salva snapshot del file YAML ───────────────────────────────────
snap = EXP_BASE / f"training_{EXP_CODE}.yaml"
if not snap.exists():
    snap.write_text(yaml.dump(cfg, sort_keys=False))
    print(f"💾 [DEBUG] Salvato snapshot → {snap.resolve()}")
else:
    print(f"ℹ️  [DEBUG] Snapshot già presente → {snap.resolve()}")

# ─── 6) Imposta seed per la riproducibilità ────────────────────────────
SEED = cfg.get("seed", 42)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
print(f"🎲 [DEBUG] Seed globale impostato a: {SEED}")


📄 [DEBUG] Config caricata da: /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/config/training.yaml
🔑 [DEBUG] EXP_CODE → 20250712175557 (fonte: TIMESTAMP)
🧬 [DEBUG] DATASET_ID         = dataset_9f30917e
🔁 [DEBUG] Folds              = [0, 1, 2]
🔒 [DEBUG] train_encoder_once = True
📂 [DEBUG] EXP_BASE → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557
💾 [DEBUG] Salvato snapshot → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/training_20250712175557.yaml
🎲 [DEBUG] Seed globale impostato a: 42


In [4]:
# ## Cell 3 – Import & Registrazione dei Trainer
# Import dinamico dei modelli definiti in YAML e verifica del registry + tipo (SSL/SL).
import sys
import importlib
from utils.training_utils.registry import TRAINER_REGISTRY

# 1) Leggi i nomi dei modelli e i loro tipi da YAML
model_cfgs = cfg["models"]
model_names = list(model_cfgs.keys())
print(f"🔄 [DEBUG] Modelli configurati in YAML: {model_names}")

# 2) Import dinamico di ciascun modulo trainers.{model}
for name in model_names:
    module_name = f"trainers.{name}"
    try:
        if module_name in sys.modules:
            importlib.reload(sys.modules[module_name])
            print(f"✅ [DEBUG] Ricaricato {module_name}")
        else:
            importlib.import_module(module_name)
            print(f"✅ [DEBUG] Importato {module_name}")
    except ImportError as e:
        print(f"❌ [DEBUG] Errore importazione {module_name}: {e}")

# 3) Verifica che tutti i modelli siano registrati e mostra il loro tipo
missing = [n for n in model_names if n not in TRAINER_REGISTRY]
if missing:
    print(f"❌ [DEBUG] Trainer mancanti nel registry: {missing}")
else:
    print("📚 [DEBUG] Trainer registry contiene e corrispondenti tipi:")
    for name in model_names:
        # Usa solo il type dal file YAML per ogni modello
        ttype = model_cfgs[name].get("type", "unknown").upper()
        print(f"  • {name:<12} → {ttype}")

# 3) Verifica funzioni in sottomoduli
import importlib, pkgutil, inspect

print("🔎 [DEBUG] Sotto-moduli caricati da utils.training_utils:")
for mod in ("registry", "device_io", "data_utils", "model_utils", "metrics"):
    m = importlib.import_module(f"utils.training_utils.{mod}")
    print(f"  • {mod:<12} → functions:", [n for n, o in inspect.getmembers(m) if inspect.isfunction(o)])

🔄 [DEBUG] Modelli configurati in YAML: ['simclr', 'jepa', 'moco_v2', 'rotation', 'supervised', 'transfer']
✅ [DEBUG] Importato trainers.simclr
✅ [DEBUG] Importato trainers.jepa
✅ [DEBUG] Importato trainers.moco_v2
✅ [DEBUG] Importato trainers.rotation
✅ [DEBUG] Importato trainers.supervised
✅ [DEBUG] Importato trainers.transfer
📚 [DEBUG] Trainer registry contiene e corrispondenti tipi:
  • simclr       → SSL
  • jepa         → SSL
  • moco_v2      → SSL
  • rotation     → SSL
  • supervised   → SL
  • transfer     → SL
🔎 [DEBUG] Sotto-moduli caricati da utils.training_utils:
  • registry     → functions: ['register_trainer']
  • device_io    → functions: ['choose_device', 'get_latest_checkpoint', 'load_checkpoint', 'load_json', 'save_checkpoint', 'save_joblib', 'save_json']
  • data_utils   → functions: ['build_loader', 'count_samples', 'default_transforms', 'discover_classes', 'extract_labels_from_keys', 'load_classifier', 'load_features', 'parse_label_from_filename', 'save_classifier

In [5]:
# %% -------------------------------------------------------------------- #
# Cell 4 – Helper utilities (Tee, paths, selezione, …)                    #
# ----------------------------------------------------------------------- #
import contextlib
import sys
import time
import math
import inspect
from pathlib import Path
from typing import Any

from utils.training_utils.registry import TRAINER_REGISTRY
from utils.training_utils.device_io import (
    get_latest_checkpoint,
    load_checkpoint,
    save_json,
    save_joblib,
)
from utils.training_utils.data_utils import count_samples
from trainers.train_classifier import train_classifier
from utils.training_utils.metrics import apply_temperature_scaling


class _Tee:
    """
    Classe per duplicare lo stdout/stderr su più target (es. console + file).
    """
    def __init__(self, *tgts):
        self.tgts = tgts

    def write(self, data: str):
        for t in self.tgts:
            t.write(data)
            t.flush()

    def flush(self):
        for t in self.tgts:
            t.flush()


def _global_experiments_append(line: str):
    """
    Appende una riga al file esperimenti globale (esperiments.md).
    """
    global_file = EXP_BASE.parent.parent / "experiments.md"
    with open(global_file, "a") as f:
        f.write(line.rstrip() + "\n")


# %% ----------------------------------------------------------------------- #
# Cell 4 – Helper utilities (Tee, paths, selezione, …)                    #
# ------------------------------------------------------------------------ #
import os
from pathlib import Path
from typing import Any

from utils.training_utils.registry import TRAINER_REGISTRY
from utils.training_utils.device_io import (
    get_latest_checkpoint,
    load_checkpoint,
    save_json,
    save_joblib,
)
from utils.training_utils.data_utils import count_samples
from trainers.train_classifier import train_classifier
from utils.training_utils.metrics import apply_temperature_scaling


def _paths(cfg: dict, model: str, fold: int) -> dict[str, Path]:
    """
    Costruisce tutti i path di output (training, inference, explain, aggregate, ecc.)
    per uno specifico modello e fold, sempre ancorati a PROJECT_ROOT.
    """
    # Parametri di formattazione iniziali
    ph = {
        'dataset_id': cfg['data']['dataset_id'],
        'exp_code': cfg['exp_code'],
        'model_name': model,
        'fold_idx': fold,
    }
    # 1) exp_dir relativo
    rel_exp_dir = cfg['output']['exp_dir'].format(**ph)
    ph['exp_dir'] = rel_exp_dir
    # 2) exp_model_dir relativo (usa ph['exp_dir'])
    rel_exp_model_dir = cfg['output']['exp_model_dir'].format(**ph)

    # Costruisci path assoluti sotto PROJECT_ROOT
    exp_dir       = (PROJECT_ROOT / rel_exp_dir)
    exp_model_dir = (PROJECT_ROOT / rel_exp_model_dir)

    # Directory per training, inference, explain, aggregate, experiment level
    tr = exp_model_dir / f"fold{fold}" / "training"
    inf = exp_model_dir / f"fold{fold}" / "inference"
    ex = exp_model_dir / f"fold{fold}" / "explain"
    ag = exp_model_dir / "_aggregate"
    el = exp_dir / "_experiment_level"

    # Creazione cartelle
    for d in (tr, inf, ex, ag, el):
        d.mkdir(parents=True, exist_ok=True)

    # Ritorna i path assoluti per tutti gli artefatti
    return {
        'ckpt_dir':       tr.resolve(),
        'ckpt_tpl':       (tr / f"{model}_bestepoch{{epoch:03d}}_fold{fold}.pt").resolve(),
        'features':       (tr / f"{model}_features_fold{fold}.pt").resolve(),
        'features_train': (tr / f"{model}_features_train_fold{fold}.pt").resolve(),
        'features_val':   (tr / f"{model}_features_val_fold{fold}.pt").resolve(),
        'clf':            (tr / f"{model}_classifier_fold{fold}.joblib").resolve(),
        'scaler':         (tr / f"{model}_ts_scaler_fold{fold}.joblib").resolve(),
        'log':            (tr / f"{model}_train_log_fold{fold}.md").resolve(),
        'loss_json':      (tr / f"{model}_train_valid_loss_fold{fold}.json").resolve(),

        'patch_preds':    (inf / f"{model}_patch_preds_fold{fold}.pt").resolve(),
        'patient_preds':  (inf / f"{model}_patient_preds_fold{fold}.csv").resolve(),
        'mc_logits':      (inf / f"{model}_mc_logits_fold{fold}.npy").resolve(),
        'metrics':        (inf / f"{model}_metrics_fold{fold}.json").resolve(),

        'gradcam_dir':    ex.resolve(),
        'metadata_csv':   (ex / f"{model}_metadata_gradcam_fold{fold}.csv").resolve(),

        'aggregate_metrics': (ag / f"{model}_metrics.json").resolve(),
        'aggregate_summary': (ag / f"{model}_summary_agg.jpg").resolve(),

        'comparison_json':    (el / "models_comparison.json").resolve(),
        'comparison_img':     (el / "models_comparison.jpg").resolve(),

        'readme':            (exp_dir / "README_EXPERIMENT.md").resolve(),
    }




def _completed(paths: dict[str, Path], is_ssl: bool) -> bool:
    """
    Verifica se il training + artefatti SSL sono già stati completati.
    """
    if not get_latest_checkpoint(paths["ckpt_dir"]):
        return False
    if is_ssl:
        return all(paths[k].exists() for k in (
            "features_train", "features_val", "clf", "scaler", "loss_json"
        ))
    return True


def _select_models(cfg: dict) -> dict[str, dict[str, Any]]:
    """
    Seleziona i modelli da eseguire in base a `run_models` o tutti i modelli.
    """
    wanted = cfg.get("run_models") or list(cfg["models"].keys())
    return {name: cfg["models"][name] for name in wanted}


def _init_trainer(name: str, m_cfg: dict, data_cfg: dict, ckpt_dir: Path):
    """
    Inizializza il trainer registrato per nome.
    """
    tr = TRAINER_REGISTRY[name](m_cfg, data_cfg)
    tr.ckpt_dir = ckpt_dir
    tr.m_cfg = m_cfg
    tr.data_cfg = data_cfg
    tr.is_ssl = m_cfg.get("type", "").lower() == "ssl"
    return tr

In [6]:
# %% ----------------------------------------------------------------------- #
# Cell 5 – Training loop                                                     #
# ----------------------------------------------------------------------- #
import math
import time
import inspect
from pathlib import Path

import torch
import joblib
import numpy as np

from utils.training_utils.device_io import get_latest_checkpoint, load_checkpoint, save_json
from utils.training_utils.data_utils import count_samples
from utils.training_utils.metrics import TemperatureScaler
from utils.training_utils.data_utils import extract_labels_from_keys
from trainers.train_classifier import train_classifier


def _run_full_training(trainer, paths: dict[str, Path], epochs: int):
    """
    Full training loop with per-batch logging, ETA estimation,
    and unified support for SSL and SL trainers.
    """
    is_ssl = getattr(trainer, "is_ssl", False)
    batch_size = int(trainer.m_cfg["training"]["batch_size"])

    # Compute total batches if not set
    if getattr(trainer, "batches_train", None) is None:
        n_samples = count_samples(Path(trainer.data_cfg["train"]))
        trainer.batches_train = math.ceil(n_samples / batch_size)
    total_batches = trainer.batches_train

    history: list[dict] = []
    for epoch in range(1, epochs + 1):
        start_time = time.time()
        loss_sum = 0.0
        corr_sum = 0
        seen = 0
        print(f"[fold{trainer.cfg_fold}] ── Epoch {epoch}/{epochs} ──")

        for i, batch in enumerate(trainer.train_loader, start=1):
            if i == 3:
              break
            # Support both signatures: train_step(batch) or train_step(x, y)
            sig = inspect.signature(trainer.train_step)
            result = (
                trainer.train_step(*batch)
                if len(sig.parameters) > 1
                else trainer.train_step(batch)
            )
            if len(result) == 4:
                _, loss, correct, bs = result
            else:
                loss, bs = result
                correct = 0

            loss_sum += loss * bs
            corr_sum += correct
            seen += bs

            pct = (i / total_batches) * 100
            eta = (time.time() - start_time) / i * (total_batches - i)
            msg = (
                f"[fold{trainer.cfg_fold}] Batch {i}/{total_batches} "
                f"({pct:5.1f}%) | Loss {loss_sum/seen:.4f}"
            )
            if not is_ssl:
                msg += f" | Acc {corr_sum/seen:.3f}"
            msg += f" | ETA {eta:6.1f}s"
            print(msg, flush=True)

        train_loss = loss_sum / seen
        if not is_ssl:
            val_loss, val_acc = trainer.validate_epoch()
            trainer.post_epoch(epoch, val_acc)
            history.append({
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_acc": val_acc,
            })
            print(f"[fold{trainer.cfg_fold}] Val → Loss {val_loss:.4f} | Acc {val_acc:.3f}")
        else:
            trainer.post_epoch(epoch, train_loss)
            history.append({"epoch": epoch, "train_loss": train_loss})
            print(f"[fold{trainer.cfg_fold}] Train → Loss {train_loss:.4f}")

        elapsed = time.time() - start_time
        print(f"[fold{trainer.cfg_fold}] ⏱  {elapsed:.1f}s\n")

    # Save training history
    save_json(history, paths["loss_json"])


def _resume_or_train(trainer, paths: dict[str, Path], epochs: int):
    """
    Resume from last checkpoint if available, otherwise run full training.
    """
    ckpt = get_latest_checkpoint(paths["ckpt_dir"])
    if ckpt:
        print(f"[fold{trainer.cfg_fold}] ⏩  Resuming from {ckpt.name}")
        model, optimizer = trainer.get_resume_model_and_optimizer()
        load_checkpoint(ckpt, model, optimizer)
    _run_full_training(trainer, paths, epochs)


def _ensure_ssl_artifacts(trainer, paths: dict[str, Path]):
    """
    For SSL models:
      1) Extract train and val features
      2) Train a linear probe on train features
      3) Calibrate the probe via temperature scaling on val logits
    """
    # 1) Extract train features
    if not paths["features_train"].exists():
        trainer.extract_features_to(paths["features_train"], split="train")

    # 2) Extract val features
    if not paths["features_val"].exists():
        trainer.extract_features_to(paths["features_val"], split="val")

    # 3) Train the linear classifier
    if not paths["clf"].exists():
        train_classifier(str(paths["features_train"]), str(paths["clf"]))

    # 4) Temperature scaling on validation logits
    if not paths["scaler"].exists():
        # Load validation features
        val_data = torch.load(paths["features_val"], map_location="cpu")
        X_val = val_data["features"].numpy()
        keys_val = val_data["keys"]

        # Load classifier and label encoder
        bundle = joblib.load(paths["clf"])
        clf = bundle["model"]
        le = bundle["label_encoder"]

        # Obtain logits or convert probs to logits
        if hasattr(clf, "decision_function"):
            logits = clf.decision_function(X_val)
        else:
            probs = clf.predict_proba(X_val)
            logits = np.log(probs + 1e-12)

        # Extract labels from keys
        labels = extract_labels_from_keys(keys_val, le)

        # Fit the TemperatureScaler
        scaler = TemperatureScaler().fit(logits, labels)
        joblib.dump(scaler, str(paths["scaler"]))


In [8]:
# %% -------------------------------------------------------------------- #
# Cell 6 – Modular Launch & Auto-Recover (cross-fold)                     #
# ------------------------------------------------------------------------#
from pathlib import Path
import contextlib, sys, torch, os
from utils.training_utils.device_io import get_latest_checkpoint, load_checkpoint

def launch_training(cfg: dict) -> None:
    """Esegue training + generazione artefatti per tutti i modelli e fold."""
    for name, m_cfg in _select_models(cfg).items():
        is_ssl = (m_cfg.get("type") == "ssl")
        epochs = int(m_cfg["training"]["epochs"])
        print(f"\n🚀  Modello '{name}'  ({'SSL' if is_ssl else 'SL'}) – epochs={epochs}")

        for fold in cfg["folds"]:
            # 1) Configurazione dati
            data_cfg = {
                "train": str((PROJECT_ROOT / cfg["data"]["train"]
                              .format(dataset_id=cfg["data"]["dataset_id"], fold_idx=fold))
                             .resolve()),
                "val":   str((PROJECT_ROOT / cfg["data"]["val"]
                              .format(dataset_id=cfg["data"]["dataset_id"], fold_idx=fold))
                             .resolve()),
                "test":  str((PROJECT_ROOT / cfg["data"]["test"]
                              .format(dataset_id=cfg["data"]["dataset_id"]))
                             .resolve())
            }
            if not Path(data_cfg["train"]).exists():
                print(f"[fold{fold}] ⚠️  shard train mancante → skip")
                continue

            # 2) Path output + trainer
            paths   = _paths(cfg, name, fold)
            trainer = _init_trainer(name, m_cfg, data_cfg, paths["ckpt_dir"])
            trainer.cfg_fold     = fold
            trainer.train_loader = trainer.build_loader("train")
            if hasattr(trainer, "validate_epoch"):
                trainer.val_loader = trainer.build_loader("val")

            # 3) Logging su stdout + file
            with open(paths["log"], "a") as logf, \
                 contextlib.redirect_stdout(_Tee(sys.stdout, logf)), \
                 contextlib.redirect_stderr(_Tee(sys.stderr, logf)):

                print(f"[fold{fold}] 📂  ckpt dir      → {paths['ckpt_dir'].resolve()}")
                print(f"[fold{fold}] 🏷   train shards  → {Path(data_cfg['train']).resolve()}")
                print(f"[fold{fold}] 🚀  avvio trainer  → '{name}'")

                # 3.1) Se abbiamo già tutti gli artefatti, salta interamente
                if _completed(paths, is_ssl):
                    print(f"[fold{fold}] ⚡  Artefatti già presenti → skip training + SSL pipeline")
                else:
                    # 4) SSL: encoder once su fold0 oppure training completo
                    if is_ssl and cfg.get("train_encoder_once", False) and fold > 0:
                        fold0_ckpt = get_latest_checkpoint(_paths(cfg, name, 0)["ckpt_dir"])
                        if fold0_ckpt:
                            mdl, _ = trainer.get_resume_model_and_optimizer()
                            load_checkpoint(fold0_ckpt, mdl, None)
                            print(f"[fold{fold}] ✅ encoder da fold0 → {fold0_ckpt.resolve()}")
                        else:
                            print(f"[fold{fold}] ⚠️  encoder fold0 mancante → train completo")
                            _resume_or_train(trainer, paths, epochs)
                    else:
                        _resume_or_train(trainer, paths, epochs)

                    # 5) SSL – Feature extraction, probe, T-scaling
                    if is_ssl:
                        _ensure_ssl_artifacts(trainer, paths)

                # 6) Append a esperimenti globali (anche in caso di skip)
                latest_ckpt = get_latest_checkpoint(paths["ckpt_dir"])
                if latest_ckpt:
                    base_dir = EXP_BASE.parent.parent.resolve()
                    try:
                        rel_path = os.path.relpath(str(latest_ckpt.resolve()), str(base_dir))
                    except Exception:
                        rel_path = str(latest_ckpt.resolve())
                else:
                    rel_path = "-"
                _global_experiments_append(
                    f"| {cfg['exp_code']} | {name} | fold{fold} | {epochs} | {rel_path} |"
                )

                print(f"[fold{fold}] ✅  completato\n")

# ─── Avvio automatico in Colab ───────────────────────────────────────────
if IN_COLAB:
    launch_training(cfg)
else:
    print("⏩  Ambiente locale: esegui manualmente  launch_training(cfg) per partire.")



🚀  Modello 'rotation'  (SSL) – epochs=2


/usr/local/lib/python3.11/dist-packages/webdataset/compat.py:381: UserWarning: set WebDataset(shardshuffle=...) to a positive integer or 0 or False
  warnings.warn("set WebDataset(shardshuffle=...) to a positive integer or 0 or False")


[fold0] 📂  ckpt dir      → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/rotation/fold0/training
[fold0] 🏷   train shards  → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/webdataset/fold0/train/patches-0000.tar
[fold0] 🚀  avvio trainer  → 'rotation'
[fold0] ── Epoch 1/2 ──
[fold0] Batch 1/62 (  1.6%) | Loss 1.4971 | ETA 1152.4s
[fold0] Batch 2/62 (  3.2%) | Loss 1.8998 | ETA  951.6s
[fold0] Train → Loss 1.8998
[fold0] ⏱  32.3s

[fold0] ── Epoch 2/2 ──
[fold0] Batch 1/62 (  1.6%) | Loss 6.1737 | ETA  810.1s
[fold0] Batch 2/62 (  3.2%) | Loss 4.0599 | ETA  778.0s
[fold0] Train → Loss 4.0599
[fold0] ⏱  26.4s



/usr/local/lib/python3.11/dist-packages/webdataset/compat.py:379: UserWarning: WebDataset(shardshuffle=...) is None; set explicitly to False or a number
  warnings.warn("WebDataset(shardshuffle=...) is None; set explicitly to False or a number")
Extracting features: 185it [11:48,  3.83s/it]


✅ Rotation features (train) saved → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/rotation/fold0/training/rotation_features_train_fold0.pt


...

✅ JEPA features (val) saved → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/jepa/fold2/training/jepa_features_val_fold2.pt
✅ Loaded 1452 keys and (1452, 512) features
📊 Class distribution:
Counter({np.str_('CHROMO'): 298, np.str_('ccRCC'): 297, np.str_('ONCO'): 291, np.str_('pRCC'): 283, np.str_('not_tumor'): 283})
✅ Filtered dataset: 1452 samples


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

      CHROMO       0.91      0.82      0.86        60
        ONCO       0.85      0.69      0.76        58
       ccRCC       0.66      0.78      0.71        59
   not_tumor       0.61      0.54      0.57        57
        pRCC       0.75      0.91      0.83        57

    accuracy                           0.75       291
   macro avg       0.76      0.75      0.75       291
weighted avg       0.76      0.75      0.75       291

Confusion Matrix:
 [[49  7  0  1  3]
 [ 2 40  3  5  8]
 [ 3  0 46 10  0]
 [ 0  0 20 31  6]
 [ 0  0  1  4 52]]
💾 Classifier saved to /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/jepa/fold2/training/jepa_classifier_fold2.joblib
[fold2] ✅  completato


🚀  Modello 'supervised'  (SL) – epochs=2
[fold0] 📂  ckpt dir      → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/su

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 143MB/s]


[fold0] 📂  ckpt dir      → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/transfer/fold0/training
[fold0] 🏷   train shards  → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/webdataset/fold0/train/patches-0000.tar
[fold0] 🚀  avvio trainer  → 'transfer'
[fold0] ── Epoch 1/2 ──
[fold0] Batch 1/185 (  0.5%) | Loss 1.6698 | Acc 0.000 | ETA 1968.2s
[fold0] Batch 2/185 (  1.1%) | Loss 1.6536 | Acc 0.000 | ETA 1890.1s
[fold0] Val → Loss 1.6085 | Acc 0.181
[fold0] ⏱  538.6s

[fold0] ── Epoch 2/2 ──
[fold0] Batch 1/185 (  0.5%) | Loss 1.6160 | Acc 0.000 | ETA 1981.4s
[fold0] Batch 2/185 (  1.1%) | Loss 1.6032 | Acc 0.062 | ETA 1913.0s
[fold0] Val → Loss 1.6009 | Acc 0.258
[fold0] ⏱  530.3s

[fold0] ✅  completato

[fold1] 📂  ckpt dir      → /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250712175557/transfer/fold1/training
[fo

In [9]:
from pathlib import Path

# Configura esperimento
EXP_CODE = "20250709142129"
ROOT = Path("/content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project")
EXP_DIR = ROOT / "data/processed/dataset_9f30917e/experiments" / EXP_CODE

# Modelli
SSL_MODELS = {"simclr", "rotation", "moco_v2", "jepa"}
SL_MODELS  = {"supervised", "transfer"}
ALL_MODELS = SSL_MODELS | SL_MODELS
N_FOLDS = 2

# File attesi per tipo di modello
FILES_PER_MODEL = {
    "SSL": [
        "{model}_bestepoch*_fold{i}.pt",
        "{model}_train_log_fold{i}.md",
        "{model}_train_valid_loss_fold{i}.json",
    ],
    "SL": [
        "{model}_bestepoch*_fold{i}.pt",
        "{model}_train_log_fold{i}.md",
        "{model}_train_valid_loss_fold{i}.json",
    ],
}

missing = []

for model in sorted(ALL_MODELS):
    model_type = "SSL" if model in SSL_MODELS else "SL"
    folds = [0] if model_type == "SSL" else list(range(N_FOLDS))

    for i in folds:
        train_dir = EXP_DIR / model / f"fold{i}" / "training"
        for pattern in FILES_PER_MODEL[model_type]:
            pattern_path = pattern.format(model=model, i=i)
            matched = list(train_dir.glob(pattern_path))  # usa direttamente il pattern
            if not matched:
                missing.append(str(train_dir / pattern_path))

# Stampa risultato
print("📂 Verifica artefatti TRAINING\n")
if not missing:
    print("✅ Tutti i file richiesti sono presenti.")
else:
    print(f"❌ Mancano {len(missing)} artefatti TRAINING:\n")
    for m in missing:
        print(" •", m)


📂 Verifica artefatti TRAINING

❌ Mancano 11 artefatti TRAINING:

 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250709142129/jepa/fold0/training/jepa_bestepoch*_fold0.pt
 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250709142129/moco_v2/fold0/training/moco_v2_bestepoch*_fold0.pt
 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250709142129/moco_v2/fold0/training/moco_v2_train_log_fold0.md
 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250709142129/moco_v2/fold0/training/moco_v2_train_valid_loss_fold0.json
 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/processed/dataset_9f30917e/experiments/20250709142129/rotation/fold0/training/rotation_bestepoch*_fold0.pt
 • /content/drive/MyDrive/ColabNotebooks/wsi-ssrl-rcc_project/data/proce